# Creating a Digital Twin (or just a executive assistant) using only local AI on a Macbook Pro

Doing this project has been interesting coming from a basis of not having written python for a while and only really having a very high level understanding of how AI works, the paradigms, language, terminology, and so on.

I made lots of mistakes, and taken the wrong path on a line of thought, more than I could ever document, and I've learned so much.

It's been an interesting journey, of a full week of working on this after work, until 2am, so 40 hours from scratch of "whats a jupyter nodebook, how do I do that locally?"

I learn through just running at the thing sideways trying to make it happen, I don't pretend to know this all, I'll be doing some crazy things here that are bad, please tell me!

## Why local?

1. because I can? (or at least that was the assumption)
1. I work in uk government, much like any corporate environment, it would not be unreasonable for them to be concerned with me sending heaps of data off to a third party provider whom they have no commercial relationship with, and not also not look to get access to any data that my normal human account couldn't do, or indeed use the in built search tools for example.
1. Having played a bit with the cloud vendors offerings, while fabulous and abstract a lot of the heavy lifting I've had to do here, I had no idea how much 40 hours of me playing around and getting things wrong might cost; in fact estimating any AI endevour appears to be a very fluid position in the industry right now, and seems to be focused on cost optimisation (after you've got your first bill presumably).

## Ambitions/use cases

I want a thing that will let me ask questions like:

- Has my access to use gov uk forms been approved?
- When was the mailchimp catchup
- What does my agenda for today look like?
- Am I free at 4pm next thursday?
- Who am I seeing today?
- Where am I meant to be today?
- How long will it take me to get to work
- Who was I talking to about snowflake?
- Send Jason an email, to tell him that I'm running late 5 minutes late
- Search my email and chats to tell me what the 3pm meeting in my calendar is about?
- Do I need an umberella today?
- What time do I need to leave for work to get to my first meeting?
- Will I make it between all my meetings today?
- Did I set the bugular alarm at home?
- If my wife isn't home, please set the bugular alarm to away, otherwise set it to home

Necessary integrations to do that:
- ✅ Google Calendar
- ✅ Google Mail
- ❎ Google Chat
- ❎ Google Maps
- ❎ Apple Messages
- ❎ Apple Notes
- ❎ Home Assistant
- ❎ WhatsApp
- ❎ Date/time converter (e.g. `today` > becomes `2024-04-22`)
- ❎ Current/forecast weather based on location
- ❎ Math
- ❎ Python execution, in a sandbox somewhere, maybe docker?
- ❎ A knowledge base of information, e.g. where my office is, my name, my email address, my phone number, etc


## Lets Go

First thing I learned is that you'll need an LLM running

I'm using [Ollama](https://www.ollama.com/) to run my llms, think of it a bit like Docker for your llms, they've packaged them up in something that vaguely resembels a Dockerfile OSI model, downloads, pushes them, runs them etc, and presents them as a HTTP interface.

When its idle its doing very little, so its safe to leave running all the time, it'll load the model in to memory, run your query and then shuffle it back out of ram when the query is done.

In [1]:
%%script false --no-raise-error
%%bash
brew install ollama

brew services start ollama
rm -f token.json chromadb

In [2]:
%load_ext autoreload
%autoreload 2

### Models.

AI has models, this is probably common enough knowledge now given everyone knows about GPT-3, GPT3.5, GPT4, GPT4o and so on, it even gets mainstream press.

The thing that wasn't so immediately clear to me was really how they fit, apart from theres ones you can run via a cloud service, like openai/azure/google/bedrock/etc and others that are 'open source' which is a peculiar language in debate, since they're not really open source, not even the rules, weighting, configuration that generated them are open source. They're just freely available to download and use, but as a binary object.

The most capable model at time of writing is from Meta `llama3:70b` which is about a 40gb download, and even on a fancy Macbook isn't amazingly responsive, especially if you're used to the speed of chatGPT, it in reviews and stats seems to track at somewhere between GPT3.5 and GPT4 in terms of capability and comprehenson. The stats are nauanced and I'm not really yet sure how they track to the quality of output of what I want.

All I'm generally seeing is fast = not so good, slow = better quality response.

On the whole I'm going to use `llama3:8b` for querying which is a fair bit smaller, and faster, but still pretty good.

I'd like to be able to experiment with smaller/faster llms for some work perhaps, but thats maybe later optimization.I

The other thing to consider with models, is some of them only run on nvidia GPUs, some only on CPUs. Given I've got a macbook, I would like to take advantage of the onboard hardware acceleration, which does somewhat limit my choices.

### Download models.

Now you'll need to pull down some models, the name/tagging is similar to docker of `name:tag`, and the tags are mutable, but provide a hash, theres caching so you can have tweaked/tuned models.

In [55]:
%%bash
ollama pull llama2:7b
ollama pull nomic-embed-text:latest
ollama pull mxbai-embed-large:latest
ollama pull all-minilm:latest
ollama pull llama3:8b
ollama pull llama3:70b
ollama pull znbang/bge:large-en-v1.5-q4_k_m
ollama pull all-minilm:l6-v2

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling 8934d96d3f08... 100% ▕████████████████▏ 3.8 GB                         
pulling 8c17c2ebb0ea... 100% ▕████████████████▏ 7.0 KB                         
pulling 7c23fb36d801... 100% ▕████████████████▏ 4.8 KB                         
pulling 2e0493f67d0c... 100% ▕████████████████▏   59 B                         
pulling fa304d675061... 100% ▕████████████████▏   91 B                         
pulling 42ba7f8a01dd... 100% ▕████████████████▏  557 B                         
verifying sha256 digest 
writing manifest 
removing any unused l

### Show installed models.

In [56]:
%%bash
ollama list

NAME                           	ID          	SIZE  	MODIFIED       
znbang/bge:large-en-v1.5-q4_k_m	78c3bcf26504	207 MB	8 seconds ago 	
all-minilm:l6-v2               	1b226e2802db	45 MB 	8 seconds ago 	
llama3:70b                     	786f3184aec0	39 GB 	9 seconds ago 	
llama3:8b                      	365c0bd3c000	4.7 GB	9 seconds ago 	
all-minilm:latest              	1b226e2802db	45 MB 	10 seconds ago	
mxbai-embed-large:latest       	468836162de7	669 MB	11 seconds ago	
nomic-embed-text:latest        	0a109f422b47	274 MB	11 seconds ago	
llama2:7b                      	78e26419b446	3.8 GB	12 seconds ago	
snowflake-arctic-embed:latest  	21ab8b9b0545	669 MB	3 days ago    	
cns:latest                     	2572851ad0e7	39 GB 	7 days ago    	
llama2:latest                  	78e26419b446	3.8 GB	11 days ago   	
m:latest                       	39704355e5c9	39 GB 	11 days ago   	


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### Show running models 
(this should be empty)

In [57]:
%%bash
ollama ps

NAME      	ID          	SIZE 	PROCESSOR	UNTIL              
llama3:70b	786f3184aec0	41 GB	100% GPU 	4 minutes from now	


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### Setup python env

In [6]:
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.huggingface_optimum import OptimumEmbedding
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.vector_stores.types import MetadataInfo, VectorStoreInfo
import pandas as pd
from IPython.display import Markdown, display
import chromadb
from datetime import datetime
from llama_index.core import (
    VectorStoreIndex,
    Settings,
    StorageContext,
    load_index_from_storage,
    Document,
)
from dateutil.relativedelta import relativedelta
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
import json
from llama_index.core.llms import ChatMessage
from llama_index.core.agent import ReActAgent
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool
from llama_index.tools.google.calendar.base import GoogleCalendarToolSpec
from llama_index.tools.google.gmail.base import GmailToolSpec
from llama_index.core.tools.tool_spec.load_and_search.base import LoadAndSearchToolSpec

from llama_index.core.tools import QueryEngineTool, ToolMetadata
from functools import reduce

import helpers as h

### We'll need a database?!

AI/LLMs benefit from using a database, apparently, sometimes at least.

The preferential database for LLMS appears to be vertex/graph databases rather than a key value, rdms, document store or other.
This allows them I think to assimilate/index the data better and then navigate it. (more on indexing in a minute)

We're going to use Chroma for this, which is a python thing that persists to sqlite, so its nice and runs in this thread without needing to add more processes and tools.

In [7]:
db = chromadb.PersistentClient(path="./chroma_db")

### Setup embedding

To create the index, we need to 'embed' the documents, this creates the links between them, which does some sentence splitting and other stuff so that the model can follow the sentiment and create links.

All seems very clever, I'll just accept its magic and somehow all works

Theres heaps of models, anidotally, some are fast, and optimized to work on my macbook, some only work on nvidia etc just the same as the other models.

what it stores is an array with lots of numbers like. The count is set on the `chunk_size` which is `1024` by default
```json
[0.042785655707120895, -0.02285182662308216, 0.04378447309136391]
```

In [8]:
Settings.embed_model = OptimumEmbedding(folder_name="./bge_onnx_large")
# Settings.embed_model = OptimumEmbedding(folder_name="./bge_onnx_base")
# Settings.embed_model = OllamaEmbedding(model_name="znbang/bge:large-en-v1.5-q4_k_m")
# Settings.embed_model = OllamaEmbedding(model_name="mxbai-embed-large")
# Settings.embed_model = OllamaEmbedding(model_name="llama2")
# Settings.embed_model = OllamaEmbedding(model_name="snowflake-arctic-embed")

## Increase that default chunk size to 2048 since some of the docs are too long

In [9]:
Settings.chunk_size = 2048

### Setup the index
I'm going to create an index for my mail, and another for my calendar at this point rather than them being indexed together; I tried this, ultimately the more data you give the poorer accuracy my results were.

In [10]:
def getIndex(CollectionName):
    chroma_collection = db.get_or_create_collection(CollectionName)
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    index = VectorStoreIndex.from_vector_store(
        vector_store, storage_context=storage_context
    )
    return index

cal_index = getIndex("calendar")
mail_index = getIndex("mail")

### Setup default LLM
Set the llm that be used if you don't manually specify one

In [11]:
Settings.llm = Ollama(model="llama3:8b", request_timeout=360.0, num_predict=-2, temperature=0.7)

### Get authenticated to Google

In [ ]:
h._get_credentials()

## Get the calender events

In [58]:
start_date = datetime.now() - relativedelta(months=3)
end_date = datetime.now() + relativedelta(months=3)

calevents = h.load_cal(start_date=start_date, end_date=end_date, number_of_results=2500)

display({"Number of Calendar Events": len(calevents)})

{'Number of Calendar Events': 448}

## Index Cal events

In [15]:
cal_index.insert_nodes(calevents, show_progress=True)

Generating embeddings:   0%|          | 0/440 [00:00<?, ?it/s]

## Get emails

In [17]:
start_date = datetime.now() - relativedelta(months=3)

emails = h.load_email(start_date=start_date, number_of_results=10000)

display({"Number of emails": len(emails)})

{'Number of emails': 1071}

## Index emails

In [18]:
mail_index.insert_nodes(emails, show_progress=True)

Generating embeddings:   0%|          | 0/1071 [00:00<?, ?it/s]

### Explain the metadata to the LLM
I've not really got this to work reliably if at all, so its marked as skip

In [19]:
%%script false --no-raise-error
vector_store_info = VectorStoreInfo(
    content_info="calendar events",
    metadata_info=[
        MetadataInfo(
            name="Summary",
            type="str",
            description=("The title of the event, e.g. 'Catch up with Mr. Smith'"),
        ),
        MetadataInfo(
            name="Start",
            type="str",
            description=(
                "the start time of the event, e.g. '2024-04-02T15:15:00+01:00'"
            ),
        ),
        MetadataInfo(
            name="End",
            type="str",
            description=("the end time of the event, e.g. '2024-04-02T15:15:00+01:00'"),
        ),
    ],
)
retriever = VectorIndexAutoRetriever(
    cal_index,
    vector_store_info=vector_store_info,
)
display(retriever.retrieve("tell me about the Mailchimp catch up"))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Ask Some Questions!
I'm probably missing a trick, but i'm not getting great relevancy or accuracy with this

In [ ]:
query_engine = cal_index.as_query_engine(similarity_top_k=20, num_predict=-2)
response = query_engine.query("mailchimp catchup")
display(response.response)

df = pd.DataFrame.from_records(
    data=list(map(lambda node: node.metadata, response.source_nodes))
)
display(df.head())

# Hard query the index without an llm
You can however skip using an LLM and just query the database, this tends to be pretty quick, and the results are much more accurate, though will probably be exact

You do need to manually embed your query first to give the db

In [ ]:
embedded_query = Settings.embed_model.get_query_embedding("mailchimp")
result = cal_index.vector_store._collection.query(
    query_embeddings=embedded_query, n_results=20
)
df = pd.DataFrame.from_records(data=result["metadatas"][0])
display(df.head())

#### Search through the mail index too

In [ ]:
embedded_query = Settings.embed_model.get_query_embedding("forms access")
result = mail_index.vector_store._collection.query(
    query_embeddings=embedded_query, n_results=10
)
df = pd.DataFrame.from_records(data=result["metadatas"][0])
display(df.head())

### Make some tools for our agent

There is a tool specification, and llamaindex have done a great job of making it super easy to do so that the llm is prepped on how to interact with it in a standard format

So we can make some tools that will use the LLM to make the query

In [23]:
cal_engine = cal_index.as_query_engine(similarity_top_k=10)
mail_engine = mail_index.as_query_engine(similarity_top_k=10)

cal_query_tool_cal = QueryEngineTool.from_defaults(
    query_engine=cal_engine,
    name="calendar",
    description=(
        f"Provides information about the user's calendar events for the past three months in the future"
    ),
)
mail_query_tool_mail = QueryEngineTool.from_defaults(
    query_engine=mail_engine,
    name="email",
    description=(f"Provides emails to and from the user for the past three months"),
)

### Vector search with a frontend LLM
And we can also make some tools that will run a quick search, and then use the LLM to answer the question based on the returned context, this seems to work well, but i don't really understand why it's not good enough to just ask the llm to do the hard work to start with

In [24]:
def vector_search(search_terms, number_of_results_to_return=20, index=None):
    embedded_query = Settings.embed_model.get_query_embedding(search_terms)
    documents = index.vector_store._collection.query(
        query_embeddings=embedded_query, 
        n_results=cal_index.vector_store._collection.count(), include=["documents", "metadatas"]
    )
    
    return documents["documents"][0][:number_of_results_to_return], documents["metadatas"][0][:number_of_results_to_return]

In [ ]:
_, metadatas = vector_search("mailchimp catchup", index=cal_index)
display(pd.DataFrame.from_records(metadatas).head())

In [ ]:

_, metadatas = vector_search("gov uk forms publisher", index=mail_index)
display(pd.DataFrame.from_records(metadatas).head())

In [27]:
def calsearch(
    search_terms, number_of_results_to_consider=5, question="summarize the output"
):
    """
    Searches the calendar for relevant documents using the provided search terms and then uses a language model to create a summary based on a provided question.

    This method performs a vector-based search to find relevant documents and then constructs a conversation for a large language model (LLM) to generate a response, effectively summarizing the search results based on the given question.

    Parameters:
    - search_terms (str): The search terms used to find relevant documents in the calendar.
    - number_of_results_to_consider (int, optional): The number of top search results to consider for summarization. Defaults to 5.
    - question (str, optional): A question or instruction to guide the summarization process. Defaults to "summarize the output".

    Returns:
    - str: The content of the message generated by the LLM that summarizes the top search results.

    Raises:
    - Exception: If an error occurs during the search or during interaction with the LLM.
    """
    documents, _ = vector_search(
        search_terms=search_terms,
        number_of_results_to_return=number_of_results_to_consider,
        index=cal_index,
    )
    messages = [
        ChatMessage(
            role="system",
            content=f"Provide the most relevant responses available to the question:{question}. Given only the calendar search results provided and no prior knowledge",
        ),
        ChatMessage(role="user", content="\n --- \n ".join(documents)),
    ]
    return Settings.llm.chat(messages).message.content


display(Markdown(calsearch("mailchimp", question="when is the mailchimp catchup?")))

Based on the provided calendar search results, the most relevant responses to the question "when is the Mailchimp catchup?" are:

1. **2024-06-11T15:00:00+01:00**: This event has a summary of "Mailchimp catch up" and is confirmed.
2. **2024-05-14T11:30:00+01:00**: This event has a summary of "CMP next steps" which might be related to Mailchimp, but it's not explicitly mentioned as a catchup.

Please note that there are no other events with the exact phrase "Mailchimp catchup" in their summaries.

In [28]:
# def mail_vector_search(search_terms, number_of_results_to_return=20):
#     embedded_query = Settings.embed_model.get_query_embedding(search_terms)
#     documents = mail_index.vector_store._collection.query(
#         query_embeddings=embedded_query, 
#         n_results=cal_index.vector_store._collection.count(), include=["documents", "metadatas"]
#     )
    
#     return documents["documents"][0][:number_of_results_to_return], documents["metadatas"][0][:number_of_results_to_return]

# _, metadatas = cal_vector_search("mailchimp catchup")
# display(pd.DataFrame.from_records(metadatas).head())

In [ ]:
def mailsearch(
    search_terms,
    number_of_results_to_consider=10,
    question="summarize the each message",
):
    """
    Searches an email index for messages that match the given search terms, then uses a language model to summarize each message.

    The function conducts a vector search to retrieve relevant email messages. It then constructs a conversation for the language model to process, asking it to summarize the content of the messages based on the provided question.

    Parameters:
    - search_terms (str): Keywords or phrases to query the email index.
    - number_of_results_to_consider (int, optional): The number of top search results to include for summarization. Defaults to 10.
    - question (str, optional): Instruction for the language model to frame the summarization. Defaults to "summarize each message".

    Returns:
    - str: A summary response generated by the language model based on the search results and the question provided.

    Raises:
    - Exception: If any issue arises during the search or the language model interaction.
    """
    documents, _ = vector_search(
        search_terms=search_terms,
        number_of_results_to_return=number_of_results_to_consider,
        index=mail_index,
    )
    messages = [
        ChatMessage(
            role="system",
            content=f"Provide the most relevant response available to the question:{question}. Given only the email search results provided and no prior knowledge",
        ),
        ChatMessage(role="user", content="\n --- \n ".join(documents)),
    ]
    return Settings.llm.chat(messages).message.content


mailsearch(
    "gov uk forms publisher",
    question="has Chris's publisher access to use gov uk forms access been approved?",
)

### Time to combine all our robots into some kind of power ranger agent 🤖 >🕵️

So now we're in a world, where we can put these tools together, and ask a LLM a question and it'll figure out what tools (might be other llms) to use to help it answer that, keep iterating, until it gets an answer.

Neat

First lets define some Basic math tools should it need it

In [30]:
def multiply(*args):
    """Multiply two or more integers and returns the result integer"""
    return reduce(lambda x, y: x * y, args, 1)


def subtract(*args):
    """subtract one or more integers from the first and returns the result integer"""
    return reduce(lambda x, y: x - y, args, 1)


def sum(*args):
    """add or sum all the integers provided and returns the result integer"""
    return reduce(lambda x, y: x - y, args, 1)


multiply_tool = FunctionTool.from_defaults(fn=multiply)
sum_tool = FunctionTool.from_defaults(fn=sum)
subtract_tool = FunctionTool.from_defaults(fn=subtract)
calsearch_tool = FunctionTool.from_defaults(fn=calsearch)
mailsearch_tool = FunctionTool.from_defaults(fn=mailsearch)

### Remove some tools from the provided packages from llamaindex
I don't want my agent to actually hit send on emails, i'm also not yet confident I want it to create events, since it can invite people to meetings, and that might be annoying

In [31]:
def remove_tool(tool_list, name):
    for tool in tool_list:
        if tool.metadata.name == name:
            tool_list.remove(tool)

### Namespace the name of the query tools
Annoyingly theres no namespacing on the tools, so using things from llamaindex can result in an overlap if you want your agent to have both capabilities!

In [32]:
def namespace_a_query_tool_list(tool_list, namespace):
    for tool in tool_list:
        tool.metadata.name = (
            f"{namespace}_{tool.metadata.name}"
            if not tool.metadata.name.startswith(f"{namespace}_")
            else tool.metadata.name
        )
    return tool_list

### Load our tool chain

In [34]:
query_engine_tools = []

query_engine_tools += [sum_tool]
query_engine_tools += [multiply_tool]

query_engine_tools += [calsearch_tool]
query_engine_tools += [mailsearch_tool]
GoogleCalendarTool = GoogleCalendarToolSpec()
GmailTool = GmailToolSpec()
query_engine_tools += namespace_a_query_tool_list(
    GoogleCalendarTool.to_tool_list(), "gcal"
)
query_engine_tools += namespace_a_query_tool_list(GmailTool.to_tool_list(), "gmail")

remove_tool(query_engine_tools, "gcal_load_data")
remove_tool(query_engine_tools, "gcal_create_event")
remove_tool(query_engine_tools, "gmail_send_draft")
remove_tool(query_engine_tools, "gmail_load_data")

# print("Wrapping " + gsearch_tools[0].metadata.name)
# gsearch_load_and_search_tools = LoadAndSearchToolSpec.from_defaults(
#     gsearch_tools[0],
# ).to_tool_list()

# print("Wrapping gmail " + gmail_tools[0].metadata.name)
# gmail_load_and_search_tools = LoadAndSearchToolSpec.from_defaults(
#     gmail_tools[0],
# ).to_tool_list()

# print("Wrapping google calendar " + gcal_tools[0].metadata.name)
# gcal_load_and_search_tools = LoadAndSearchToolSpec.from_defaults(
#     gcal_tools[0],
# ).to_tool_list()

df = pd.DataFrame.from_records(
    data=[tool.metadata.__dict__ for tool in query_engine_tools],
    index=["name", "description", "fn_schema", "return_direct"],
)
display(df)

,,,
name,description,fn_schema,return_direct
sum,sum(*args)\nadd or sum all the integers provided and returns the result integer,<class 'pydantic.v1.main.sum'>,False
multiply,multiply(*args)\nMultiply two or more integers and returns the result integer,<class 'pydantic.v1.main.multiply'>,False
calsearch,"calsearch(search_terms, number_of_results_to_consider=5, question='summarize the output')\n\n Searches the calendar for relevant documents using the provided search terms and then uses a language model to create a summary based on a provided question.\n\n This method performs a vector-based search to find relevant documents and then constructs a conversation for a large language model (LLM) to generate a response, effectively summarizing the search results based on the given question.\n\n Parameters:\n - search_terms (str): The search terms used to find relevant documents in the calendar.\n - number_of_results_to_consider (int, optional): The number of top search results to consider for summarization. Defaults to 5.\n - question (str, optional): A question or instruction to guide the summarization process. Defaults to ""summarize the output"".\n\n Returns:\n - str: The content of the message generated by the LLM that summarizes the top search results.\n\n Raises:\n - Exception: If an error occurs during the search or during interaction with the LLM.\n",<class 'pydantic.v1.main.calsearch'>,False
mailsearch,"mailsearch(search_terms, number_of_results_to_consider=10, question='summarize the each message')\n\n Searches an email index for messages that match the given search terms, then uses a language model to summarize each message.\n\n The function conducts a vector search to retrieve relevant email messages. It then constructs a conversation for the language model to process, asking it to summarize the content of the messages based on the provided question.\n\n Parameters:\n - search_terms (str): Keywords or phrases to query the email index.\n - number_of_results_to_consider (int, optional): The number of top search results to include for summarization. Defaults to 10.\n - question (str, optional): Instruction for the language model to frame the summarization. Defaults to ""summarize each message"".\n\n Returns:\n - str: A summary response generated by the language model based on the search results and the question provided.\n\n Raises:\n - Exception: If any issue arises during the search or the language model interaction.\n",<class 'pydantic.v1.main.mailsearch'>,False
gcal_get_date,get_date()\n\n A function to return todays date. Call this before any other functions if you are unaware of the date.\n,<class 'pydantic.v1.main.get_date'>,False
gmail_search_messages,"search_messages(query: str, max_results: Optional[int] = None)\n",<class 'pydantic.v1.main.search_messages'>,False
gmail_create_draft,"create_draft(to: Optional[List[str]] = None, subject: Optional[str] = None, message: Optional[str] = None) -> str\nCreate and insert a draft email.\n Print the returned draft's message and id.\n Returns: Draft object, including draft id and message meta data.\n\n Args:\n to (Optional[str]): The email addresses to send the message to\n subject (Optional[str]): The subject for the event\n message (Optional[str]): The message for the event\n",<class 'pydantic.v1.main.create_draft'>,False
gmail_update_draft,"update_draft(to: Optional[List[str]] = None, subject: Optional[str] = None, message: Optional[str] = None, draft_id: str = None) -> str\nUpdate a draft email.\n Print the returned draft's message and id.\n This function is required to be passed a draft_id that is obtained when creating messages\n Returns: Draft object, including draft id and message meta data.\n\n Args:\n to (Optional[str]): The email addresses to send the message to\n subject (Optional[str]): The subject for the event\n message (Optional[str]): The message for the event\n draft_id (str): the id of the draft to be updated\n",<class 'pydantic.v1.main.update_draft'>,Fals

### Create our agent

Enabling `verbose` means we see the whole conversation between the agent and its tools, it explains its 'thought' process.

Also define a nice helper function to display the results

In [35]:
myllm = Ollama(
    model="llama3:8b",
    # context_window=8096,
    num_predict=-2,
    # temperature=0.1,
)
# myllm = Ollama(model="llama3:70b")

agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=myllm,
    verbose=True,
    request_timeout=3600.0,
    max_iterations=100,
    context=h.get_system_prompt()
)


def ask(question, *args, **kwargs):
    response = agent.chat(question, *args, **kwargs)
    display(Markdown("## Response:"))
    display(response.response)
    display(Markdown("## Tools used:"))
    display(
        pd.DataFrame.from_records(
            data=[[source.tool_name, source.is_error] for source in response.sources],
            columns=["Tool", "Output"],
        )
    )

### Ask it some questions!!!

In [ ]:
ask("search my calendar, find out when the mailchimp catch up was?")

In [ ]:
ask("I would like to know if chris's gov uk forms access been approved?")

In [38]:
ask("search my calendar, to tell me what i've got on today")

Thought: I can use a tool to search your calendar.
Action: gcal_get_date
Action Input: {}
Observation: 2024-06-14
Thought: (Implicit) I can answer without any more tools!
Answer: It seems that you have no events scheduled for today, June 14th.


## Response:

'It seems that you have no events scheduled for today, June 14th.'

## Tools used:

,Tool,Output
0,gcal_get_date,False


In [39]:
ask("search my calendar to tell me who am I meeting with today?")

Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: gcal_get_date
Action Input: {'date': '2023-06-14'}
Observation: Error: GoogleCalendarToolSpec.get_date() got an unexpected keyword argument 'date'
Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: gcal_get_date
Action Input: {}
Observation: 2024-06-14
Thought: Since the date has been retrieved, I can now search for events on that specific day.
Action: calsearch
Action Input: {'search_terms': '2024-06-14', 'number_of_results_to_consider': 5, 'question': "What's scheduled for this day?"}
Observation: Scheduled for this day is:

1. Services Week 2024 opening event: This event has already started, and it's scheduled from 11:00 AM to 12:00 PM.
2. OCI Fleet Application Management & Launchpad Discussion: This event is confirmed and scheduled for 14:00-15:00 on March 26th.

Note that the other events listed are not s

## Response:

"Hi Chris, based on my calendar search, it appears that you have two events scheduled for today. The first one is the Services Week 2024 opening event, which started at 11:00 AM and is expected to end by 12:00 PM. The second event is the OCI Fleet Application Management & Launchpad Discussion, which is confirmed but not taking place today as it's scheduled for March 26th."

## Tools used:

,Tool,Output
0,gcal_get_date,True
1,gcal_get_date,False
2,calsearch,False


In [ ]:
ask("Can you draft a new email to helpdesk and support@example.com about a service outage")

In [ ]:
from llama_index.core.tools.ondemand_loader_tool import OnDemandLoaderTool
from llama_index.readers.wikipedia import WikipediaReader
from typing import List

from llama_index.readers.google import GmailReader, GoogleCalendarReader


from pydantic import BaseModel

# reader = WikipediaReader()
reader = GmailReader()
# tool = OnDemandLoaderTool.from_defaults(
#     reader,
#     name="email",
#     description="A tool for searching and retrieving emails",
# )

# tool(["Berlin"], query_str="What's the arts and culture scene in Berlin?")

query_engine_tools = [tool]
agent = ReActAgent.from_tools(
    query_engine_tools, llm=myllm, verbose=True, max_iterations=30
)

response = agent.chat(
    "do not try to answer this yourself. use your tools. Has my forms access been approved, explain why you think this"
)